In [21]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.backend import mean
from tensorflow.keras.backend import square


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import CuDNNLSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten

from tensorflow.keras.utils import Sequence
from tensorflow.keras import Input
from tensorflow.keras import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D
import numpy as np
import pickle
from scipy.stats import zscore
import datetime
import pytz



# np.random.seed(seed=11)

# with open('../../../data_GRS1915/468202_len128_s2_4cad_counts_errorfix.pkl', 'rb') as f:
#     segments = pickle.load(f)
# with open('../../../data_GRS1915/468202_len128_s2_4cad_errors_errorfix.pkl', 'rb') as f:
#     errors = pickle.load(f)

# # errors = np.expand_dims((np.squeeze(errors)/(np.max(segments, axis=1)-np.min(segments, axis=1))), axis=-1).astype(np.float32)
# # segments = np.expand_dims(((np.squeeze(segments)-np.min(segments, axis=1))/(np.max(segments, axis=1)-np.min(segments, axis=1))), axis=-1).astype(np.float32)
# # errors = ((errors)/np.std(segments)).astype(np.float32)
# # segments = zscore(segments, axis=None).astype(np.float32)  # standardize


# errors = ((errors)/np.expand_dims(np.std(segments, axis=1), axis=1)).astype(np.float32)
# segments = zscore(segments, axis=1).astype(np.float32)  # standardize per segment


def chi2(y_err):
    def MSE_scaled(y_in, y_out,):
        return mean(square(y_in-y_out)/square(y_err))
    return MSE_scaled


class DataGenerator(Sequence):
    """
    Generates data for Keras
    https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
    https://stackoverflow.com/questions/53105294/implementing-a-batch-dependent-loss-in-keras
    """
    def __init__(self, y_in, y_err, batch_size=32, shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        self.y_in = y_in
        self.y_err = y_err        
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.y_in) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Find list of IDs
        y_in = self.y_in[indexes]
        y_err = self.y_err[indexes]
        return [y_in, y_err], y_in
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.y_in))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit.
    https://www.tensorflow.org/guide/keras/custom_layers_and_models#putting_it_all_together_an_end-to-end_example"""
    
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


original_dim = 128
intermediate_dim = 512
latent_dim = 16

# Define encoder model.
original_inputs = tf.keras.Input(shape=(original_dim,1), name='Encoder_input')
input_err = Input(shape=(original_dim,1))
x = layers.CuDNNLSTM(intermediate_dim, return_sequences=False, name="Encoder_LSTM")(original_inputs)
z_mean = layers.Dense(latent_dim, name='Latent_mean')(x)
z_log_var = layers.Dense(latent_dim, name='Latent_log_variance')(x)
z = Sampling(name="Sampler")((z_mean, z_log_var))
encoder = tf.keras.Model(inputs=original_inputs, outputs=z, name='Encoder')

# Define decoder model.
latent_inputs = tf.keras.Input(shape=(latent_dim,), name='Decoder_input')
x = layers.RepeatVector(original_dim, name="Expand_to_decoder_shape")(latent_inputs)
x = layers.CuDNNLSTM(intermediate_dim, return_sequences=True, name="Decoder_LSTM")(x)
outputs = layers.TimeDistributed(layers.Dense(1),name="Collapse_to_output_shape")(x)
decoder = tf.keras.Model(inputs=latent_inputs, outputs=outputs, name='Decoder')

# Define VAE model.
outputs = decoder(z)
vae = tf.keras.Model(inputs=[original_inputs, input_err], outputs=outputs, name='vae')

# Add KL divergence regularization loss.
kl_loss = - 0.5 * tf.reduce_mean(
    z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
vae.add_loss(kl_loss)

optimizer = tf.keras.optimizers.SGD(lr=2e-5, clipvalue=0.5) #Adam(clipvalue=0.5)

vae.compile(optimizer, loss=chi2(input_err))

from keras.utils.vis_utils import plot_model
plot_model(vae, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [18]:
print(decoder.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Decoder_input (InputLayer)   (None, 16)                0         
_________________________________________________________________
Expand_to_decoder_dimensions (None, 128, 16)           0         
_________________________________________________________________
Decoder_LSTM (CuDNNLSTM)     (None, 128, 512)          1085440   
_________________________________________________________________
Collapse_to_output_dimension (None, 128, 1)            513       
Total params: 1,085,953
Trainable params: 1,085,953
Non-trainable params: 0
_________________________________________________________________
None
